In [8]:
import json
import matplotlib.pyplot as plt

import sklearn
from sklearn import metrics

import pandas as pd

In [15]:
with open('y_pred_list.json', 'r') as preds_file:
    predictions = json.load(preds_file)

with open('y_true_list.json','r') as labels_file:
    targets = json.load(labels_file)

print(predictions[0][0:5])

[3.330878462293185e-05, 0.00023926969151943922, 0.9672697186470032, 0.8805846571922302, 1.578277260705363e-05]


In [16]:
new_predictions = []
new_targets = []

for i, episode in enumerate(targets):
    new_episode_pred = []
    new_episode_targ = []
    for j, label in enumerate(episode):
        if label == 1:
            new_episode_pred.append(predictions[i][j])
            new_episode_targ.append(label)
    new_predictions.append(new_episode_pred)
    new_targets.append(new_episode_targ)
    print(new_predictions[i][:5])

predictions = new_predictions
targets = new_targets

[1.578277260705363e-05, 8.197518582164776e-06, 0.9936849474906921, 2.0059200134259214e-10, 4.959636035195558e-10]
[3.292331166449003e-05, 0.7798627614974976, 0.002737141214311123, 0.9987782835960388, 1.5985327991074882e-05]
[7.907480892299645e-09, 7.768292675791599e-08, 8.596911982294841e-08, 1.7562202003773564e-07, 0.9999316930770874]
[0.0002637580910231918, 0.13373054563999176, 4.7512617840084204e-08, 0.25004345178604126, 0.9999948740005493]
[0.0029154643416404724, 0.889114260673523, 1.3416973310143021e-08, 1.6033996530495642e-07, 8.645923976757786e-10]
[1.2339022532614763e-06, 5.722436253563501e-05, 5.731635721595296e-10, 0.9999996423721313, 0.969560444355011]
[0.0015836970414966345, 0.00040196144254878163, 4.179608126264611e-08, 0.05089742690324783, 2.900826439145021e-05]
[0.4866275191307068, 5.060452323846221e-08, 7.046960348588982e-08, 1.6420069785198083e-13, 2.2958200553320485e-13]


In [27]:
for prediction in predictions:
    for i, value in enumerate(prediction):
        if value > 0.5:
            prediction[i] = 1.0
        else:
            prediction[i] = 0.0

print(len(predictions))
for prediction in predictions:
    print(prediction)

8
[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [28]:
results = []

for prediction, target in zip(predictions,targets):
    print("Pred: ", prediction[:10])
    print("Targ: ", target[:10])
    results.append(metrics.classification_report(target, prediction, output_dict=True))

print(results[0])

Pred:  [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Targ:  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Pred:  [0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Targ:  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Pred:  [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0]
Targ:  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Pred:  [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0]
Targ:  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Pred:  [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Targ:  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Pred:  [0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Targ:  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Pred:  [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
Targ:  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Pred:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Targ:  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
{'0.0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 0}

/home/michael/Projects/whodidit/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/michael/Projects/whodidit/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/michael/Projects/whodidit/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mich

In [14]:
precision_values = 0
recall_values = 0
f1_values = 0
num_vals = len(results)

for result in results:
    weighted_avg = result['weighted avg']
    precision_values += weighted_avg['precision']
    recall_values += weighted_avg['recall']
    f1_values += weighted_avg['f1-score']

precision = precision_values / num_vals
recall = recall_values / num_vals
f1 = f1_values / num_vals

results_dict = {
    'precision': precision,
    'recall': recall,
    'f1-score': f1
}

results_df = pd.DataFrame.from_dict(results_dict, orient='index')

print(results_df)

                  0
precision  1.000000
recall     0.231487
f1-score   0.373677
